In [2]:
rawdata = spark.read.csv('s3://test-financial-news-bucket/news/crypto/2024-05-31_full_record.csv',inferSchema=True,header=True)
rawdata.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                  id|              source|            category|            headline|                href|             summary|             content|            datetime|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|14506777316744071...|            CoinDesk|            Business|Bitcoin Breaks to...|https://finance.y...|The price action ...|The price of bitc...|                null|
|      At 11:45 am ET| bitcoin was chan...|                 300| lower by 1% over...|000. The broader ...|                null|                null|                null|
|                 May|              though| has been strong ...| which is now hig...|000 level. That's...| fueled by a 31% ...|                null|  

In [4]:
articles = rawdata.toDF("id","source","category","headline","href","summary","content","datetime")
articles.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                  id|              source|            category|            headline|                href|             summary|             content|            datetime|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|14506777316744071...|            CoinDesk|            Business|Bitcoin Breaks to...|https://finance.y...|The price action ...|The price of bitc...|                null|
|      At 11:45 am ET| bitcoin was chan...|                 300| lower by 1% over...|000. The broader ...|                null|                null|                null|
|                 May|              though| has been strong ...| which is now hig...|000 level. That's...| fueled by a 31% ...|                null|  

In [7]:
articles.filter(articles['content'].isNull()).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

278

In [8]:
cleaned_articles = articles.filter(articles['content'].isNotNull())
cleaned_articles.filter(cleaned_articles['content'].isNull()).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

In [9]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

tokenizer = Tokenizer(inputCol="content", outputCol='words')
wordsData = tokenizer.transform(cleaned_articles)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20)
featurizedData = hashingTF.transform(wordsData)
featurizedData.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                  id|              source|            category|            headline|                href|             summary|             content|            datetime|               words|         rawFeatures|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|14506777316744071...|            CoinDesk|            Business|Bitcoin Breaks to...|https://finance.y...|The price action ...|The price of bitc...|                null|[the, price, of, ...|(20,[0,1,3,4,5,6,...|
|91528743924387766...|       CoinMarketCap|            Business|Bitcoin Struggles...|https://finance.y...|Bitcoin (BTC) is ...|Bitcoin Struggles...|    

In [11]:
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
rescaledData.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                  id|              source|            category|            headline|                href|             summary|             content|            datetime|               words|         rawFeatures|            features|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|14506777316744071...|            CoinDesk|            Business|Bitcoin Breaks to...|https://finance.y...|The price action ...|The price of bitc...|                null|[the, price, of, ...|(20,[0,1,3,4,5,6,...|(20,[0,1,3,4,5,6,...|
|91528743924387766...|       CoinMarketCap|            Business|Bitc

In [30]:
from pyspark.sql.types import *

schema = StructType([StructField('words', ArrayType(StringType()))])

df = spark.createDataFrame(([[["bitcoin"]]]), schema).toDF("words")
df.show()

bitcoin = hashingTF.transform(df)
bitcoin.show()

featureVec = bitcoin.select('rawFeatures').collect()
print(featureVec)

bitcoinID = int(featureVec[0].rawFeatures.indices[0])
print(bitcoinID)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+
|    words|
+---------+
|[bitcoin]|
+---------+

+---------+---------------+
|    words|    rawFeatures|
+---------+---------------+
|[bitcoin]|(20,[18],[1.0])|
+---------+---------------+

[Row(rawFeatures=SparseVector(20, {18: 1.0}))]
18

In [32]:
from pyspark.sql.types import FloatType
from pyspark.sql.functions import udf

termExtractor = udf(lambda x: float(x[bitcoinID]), FloatType())
bitcoinDF = rescaledData.withColumn('score', termExtractor(rescaledData.features))

bitcoinDF.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+
|                  id|              source|            category|            headline|                href|             summary|             content|            datetime|               words|         rawFeatures|            features|    score|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+
|14506777316744071...|            CoinDesk|            Business|Bitcoin Breaks to...|https://finance.y...|The price action ...|The price of bitc...|                null|[the, price, of, ...|(20,[0,1,3,4,5,6,...|(20,[0,1,3,4,5,6,...|2.4030955|
|91528743924387766...|      

In [35]:
sortedResults = bitcoinDF.filter("score > 0").orderBy('score', ascending=False).select('headline', 'content','score')
sortedResults.show(truncate=100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+---------+
|                                                                                            headline|                                                                                             content|    score|
+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+---------+
|                    Bitcoin Breaks to Low End of Trading Range, but June Data Could Be Next Catalyst|The price of bitcoin {{BTC}} has traded in an exceptionally tight trading range surrounding $68,0...|2.4030955|
|         Donald Trump’s latest appeal to pro-crypto voters includes asking Elon Musk for policy tips|"Newly convicted felon Donald Trump has be